#### catatan: 
beberapa data taksonomi memang kosong dari NCBInya, misal data interaksi virus pycv malah terhubung ke genus capsicum bukan spesies capsicum. dan itu memang dari data GloBInya langsung

In [1]:
# # reset import package
# def reloadPackageOwn():
#     from importlib import reload  
#     import os # we use os.path.join, os.path.basename
#     import sys # we use sys.path
#     import glob # we use glob.glob
#     import importlib # we use importlib.import_module

#     import_folder = os.getcwd()
#     sys.path.append(import_folder) # this tells python to look in `import_folder` for imports
#     for src_file in glob.glob(os.path.join(import_folder, '*.py')):
#         name = os.path.basename(src_file)[:-3]
#         importlib.import_module(name)
#         reload(sys.modules[name])
#         importlib.import_module(name)
        
# reloadPackageOwn()

In [2]:
from tqdm import tqdm
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec.embedders import FastText,Word2Vec
from pyrdf2vec.walkers import RandomWalker
from pyvis.network import Network
from sklearn.manifold import TSNE
from umap import UMAP
from SPARQLWrapper import SPARQLWrapper
from jcopml.plot import plot_missing_value

import plotly.express as px
import pandas as pd
import numpy as np
import requests
import os
import networkx as nx
import matplotlib.pyplot as plt

from modul.vectorReferenced import get_taxon_vector,cek_ncbi_id_by_wiki_id_via_string
from modul.filterNodeEdge import removeNodeAndEdgeByFilter,removeEdgesNotInNodes
from modul.helper_umum import contains_string_entire_column,contains_string_entire_column_boolean, minmax, std_scale
#from process import cek_bfs, nx_to_pyviz
from modul.grafHelper import _set_networkx_graph, _plot_nx_by_matplotlib
from modul.visualisasiHelper import embeddingPlot,plotly_graph
from modul.embeddingHelper import df_serangga_to_rdf, rdf_KG_to_embeddings, df_to_dictionary_taxon
from modul.custom_degree_centrality import degree_centrality_custom

/Users/mohzulkiflikatili/miniconda3/envs/thesis-flask/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mohzulkiflikatili/miniconda3/envs/thesis-flask/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mohzulkiflikatili/miniconda

In [3]:
ncbi_ontology_url = 'http://localhost:3030/mydataset/query'
dict(get_taxon_vector("Pepper yellow leaf curl virus",ncbi_ontology_url,False))['family']

'NCBI:10811_Geminiviridae'

#### Parameter

In [4]:
data=[
    ('begomovirus_contoh_hasil','Pepper yellow leaf curl virus','Aleyrodidae','Bemisia Tabaci'),
    ('1cucu','Cucumber mosaic virus','Aphididae','Myzus persicae'),
    ('2cri','Tomato chlorosis virus','Aleyrodidae','Bemisia Tabaci'),
    ('3wai','Maize chlorotic dwarf virus','Cicadellidae','Graminella nigrifrons'),
    ('4beg','Tomato yellow leaf curl China virus','Aleyrodidae','Bemisia Tabaci'),
    ('5pol','Cereal yellow dwarf virus','Aphididae','Schizaphis graminum'),
    ('6pea','Pea enation mosaic virus 1','Aphididae','Acyrthosiphon pisum'),
    ('7cucur','Cucurbit yellow stunting disorder virus','Aleyrodidae','Bemisia Tabaci'),
    ('8ten','Rice stripe tenuivirus','Delphacidae','Laodelphax striatellus'),
    ('9fiji','Southern rice black-streaked dwarf virus','Delphacidae','Sogatella furcifera'),
    ('10capchlo','Capsicum chlorosis orthotospovirus','Thripidae','Thrips Palmi'),
    ('11barley','Barley yellow dwarf virus GAV','Aphididae','Sitobion avenae'),
    ('12tospot','Tomato spotted wilt orthotospovirus','Thripidae','Frankliniella occidentalis'),
    ('13svyv','squash vein yellowing virus','Aleyrodidae','Bemisia Tabaci'),
    ('14sbmv','soybean mosaic virus','Aphididae','Aphis glycines'),
    ('15blv','bean leafroll virus','Aphididae','Acyrthosiphon pisum'),
    ('16rgdv','rice gall dwarf virus','Cicadellidae','Recilia dorsalis'), #sedikit
    ('17srbsdv','southern rice black-streaked dwarf virus','Delphacidae','Sogatella furcifera'),
    ('18tsrv','tomato severe rugose virus','Aleyrodidae','Bemisia tabaci'),
    ('19gbnv','groundnut bud necrosis virus','Thripidae','Thrips palmi'),
    ('20wbnv','Watermelon bud necrosis virus','Thripidae','Thrips palmi'),
    # error dibawah ini
    # ('+13Poty','Potyvirus','Aphididae','Myzus'),
    # ('+11tung','Tungrovirus','Nilaparvata','Nilaparvata'),
]

data_,nama_virus,acuan_,ujian_=data[3] # vektor acuan  #data virus
# link enpoint sparql ncbi_ontology
ncbi_ontology_url = 'http://localhost:3030/mydataset/query'

#### input data

In [5]:
#1
#baca data
df_node=pd.read_csv('dari_praproses/'+data_+'_node.csv',index_col=0) 
df_edge=pd.read_csv('dari_praproses/'+data_+'_edge.csv',index_col=0)

In [6]:
# df_node[df_node['group']=='virus']
# df_node['group'].unique()

contains_string_entire_column(df_node,'67753_Crinivirus').taxon_id.to_list()

[]

In [7]:
acuan_

'Cicadellidae'

In [8]:
# pra-proses khusus proses
# hapus serangga yg cuma famili (mengikuti acuan). soalnya klo cuma tampil famili apa gunanya?
filter_genus_sampai_species_null=(
    (df_node.genus.isnull()) &
    (df_node.species.isnull()) &
    (df_node['class']=='NCBI:50557_Insecta')
)
df_node,df_edge = removeNodeAndEdgeByFilter(df_node[filter_genus_sampai_species_null], df_node,df_edge)

removeNodeAndEdgeByFilter
sebelum : 536 719
sesudah : 507 679


In [9]:
print(len(df_edge))
df_edge.drop_duplicates(inplace=True)
print(len(df_edge))

679
679


In [10]:
# pra-proses khusus proses
#isi data kosong. mengisi takson kosong, dengan takson sebelumnya, untuk tambalan
takson=[
    'superkingdom','kingdom','phylum','class','order','family','genus','species'
]

for x,i in enumerate(takson):
    if (i!='superkingdom'): #selain superkingdom update dengan data sebelumnya
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x-1]]+'^'+i
    else: 
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x+1]]+'^'+i

In [11]:
# pra-proses khusus proses
# eksperimen tambahan. bikin fakta tambahan, yaitu relasi virus utama dengan acuan 
# virus_utama=df_node[df_node.virus_utama==True].taxon_id.to_list()
# serangga_acuan=contains_string_entire_column(df_node,acuan_).taxon_id.to_list()
# print(len(df_edge))
# for i in virus_utama:
#     for j in serangga_acuan:
#         dict = {'source_taxon_id':i,'target_taxon_id':j,'interaction_type':'pathogenOf'}
#         df_edge = pd.concat([pd.DataFrame(dict,index=[0]), df_edge], ignore_index = True)
#         # df_edge.loc[len(df_edge.index),['source_taxon_id','target_taxon_id','interaction_type']] = [i,j,'pathogenOf']
# print(len(df_edge))

#satu virus saja #perlu ini soalnya kalo tida 2cri nol
virus_utama=df_node[df_node.virus_utama==True].taxon_id.to_list()
serangga_acuan=contains_string_entire_column(df_node,acuan_).taxon_id.to_list()
print(len(df_edge))
for j in serangga_acuan:
    dict_serangga = {'source_taxon_id':virus_utama[0],'target_taxon_id':j,'interaction_type':'pathogenOf'}
    df_edge = pd.concat([pd.DataFrame(dict_serangga,index=[0]), df_edge], ignore_index = True)
    # df_edge.loc[len(df_edge.index),['source_taxon_id','target_taxon_id','interaction_type']] = [i,j,'pathogenOf']
print(len(df_edge))


679
679


In [12]:
# Ini harusnya di praproses. tapi belum fix baiknya hapus atau tidak
# hapus yang bukan virus utama, terakhir akurasi 0.85, kalau berkurang hapus saja ini
# bukan_virus_utama=(df_node['group']=="virus") & (df_node.virus_utama!=True)
# df_node,df_edge = removeNodeAndEdgeByFilter(df_node[bukan_virus_utama], df_node,df_edge)

In [13]:
if(len(df_node[df_node['group']=="serangga"])<=2):
    print("cuma dua serangga")

# Konversi graf

In [14]:
#3
#konversi graph 
gnx = _set_networkx_graph(df_node, df_edge)

# # cuma tampilan, visualisasi graf
# _plot_nx_by_matplotlib(gnx)

# visualisasi Graf

In [15]:
plotly_graph(gnx)

# # # cuma tampilan, visualisasi pyviz
# from pyvis.network import Network
# nt = Network('500px', '900px',directed=True,notebook=True)
# # nt.show_buttons(filter_=['physics'])
# nt.toggle_physics(True)

# for i,data in df_node.iterrows():
#     nt.add_node(
#         data['taxon_id'],
#         label=data['taxon_name'],
#         superkingdom=data['superkingdom'],
#         kingdom=data['kingdom'],
#         filum=data['phylum'],
#         kelas=data['class'],
#         ordo=data['order'],
#         famili=data['family'],
#         genus=data['genus'],
#         spesies=data['species'],
#         # group=data['group'],
#         color=data['color'],
#         )
    
# for i,data in df_edge.iterrows():
#     nt.add_edge(
#         data['source_taxon_id'],
#         data['target_taxon_id'],
#         label=data['interaction_type'])

# # nt.show_buttons(filter_=['physics'])
# nt.show("tmp.fig02.html")

In [16]:
# visualisasi embedding serangga

# embedding
# input : df_node, URL
URL = "http://pyRDF2Vec"
df_serangga = df_node[df_node['group']=="serangga"]
CUSTOM_KG = df_serangga_to_rdf(df_serangga, URL)
# proses
list_serangga=df_node[df_node['group']=="serangga"].taxon_id.to_list()
list_of_entities = [ URL+"#"+taxon_id for taxon_id in list_serangga ]
transformer, embeddings, _ = rdf_KG_to_embeddings(CUSTOM_KG, list_of_entities)
# sama dua2nya # list_of_entities == transformer._entities 

# dictionary serangga
dictionary_serangga = df_to_dictionary_taxon(df_serangga)
# output
# embeddings, list_of_entities, dictionary_serangga

# plotting
embeddingPlot(embeddings, list_of_entities, dictionary_serangga, 650, 650)

# Analisis Interaksi

In [17]:
# from modul.custom_degree_centrality import degree_centrality_custom
# import importlib, sys
# importlib.reload(sys.modules['modul.custom_degree_centrality'])

#4 
# Degree Centrality Custom
virus_utama_ids=list(df_node[df_node['virus_utama']==True].taxon_id)
serangga_ids=list(df_node[df_node['group']=="serangga"].taxon_id)
results_dc = degree_centrality_custom(gnx,virus_utama_ids,serangga_ids)
allnodes = gnx.nodes

Helcystogramma NCBI:689130 | degree: 1 | langsung: 0 | melalui tanaman: 1
Pamphlebia rubrolimbraria NCBI:2203055 | degree: 1 | langsung: 0 | melalui tanaman: 1
Apaustus menes NCBI:354773 | degree: 1 | langsung: 0 | melalui tanaman: 1
Parnara poutieri NCBI:218705 | degree: 1 | langsung: 0 | melalui tanaman: 1
Baoris pagana NCBI:1972205 | degree: 1 | langsung: 0 | melalui tanaman: 1
Wallengrenia premnas NCBI:2049200 | degree: 1 | langsung: 0 | melalui tanaman: 1
Parnara naso bada NCBI:674894 | degree: 2 | langsung: 0 | melalui tanaman: 2
Potanthus NCBI:218755 | degree: 1 | langsung: 0 | melalui tanaman: 1
Parnara ganga NCBI:904384 | degree: 1 | langsung: 0 | melalui tanaman: 1
Parnara guttatus NCBI:218706 | degree: 2 | langsung: 0 | melalui tanaman: 2
Estigmene senegalensis NCBI:56593 | degree: 1 | langsung: 0 | melalui tanaman: 1
Spilosoma nigricosta NCBI:214365 | degree: 2 | langsung: 0 | melalui tanaman: 2
Phragmataecia purpureus NCBI:753406 | degree: 1 | langsung: 0 | melalui tanaman

In [18]:
urutan=sorted(results_dc.items(), key=lambda item: item[1], reverse=True)
urutan

[('NCBI:689130', 0.0),
 ('NCBI:2203055', 0.0),
 ('NCBI:354773', 0.0),
 ('NCBI:218705', 0.0),
 ('NCBI:1972205', 0.0),
 ('NCBI:2049200', 0.0),
 ('NCBI:674894', 0.0),
 ('NCBI:218755', 0.0),
 ('NCBI:904384', 0.0),
 ('NCBI:218706', 0.0),
 ('NCBI:56593', 0.0),
 ('NCBI:214365', 0.0),
 ('NCBI:753406', 0.0),
 ('NCBI:1555284', 0.0),
 ('NCBI:1376690', 0.0),
 ('NCBI:236749', 0.0),
 ('NCBI:744611', 0.0),
 ('NCBI:129553', 0.0),
 ('NCBI:1892586', 0.0),
 ('NCBI:705799', 0.0),
 ('NCBI:214146', 0.0),
 ('NCBI:1424573', 0.0),
 ('NCBI:2736686', 0.0),
 ('NCBI:708061', 0.0),
 ('NCBI:859963', 0.0),
 ('NCBI:880113', 0.0),
 ('NCBI:1480102', 0.0),
 ('NCBI:988038', 0.0),
 ('NCBI:110367', 0.0),
 ('NCBI:127176', 0.0),
 ('NCBI:1113392', 0.0),
 ('NCBI:441995', 0.0),
 ('NCBI:2785647', 0.0),
 ('NCBI:856302', 0.0),
 ('NCBI:856295', 0.0),
 ('NCBI:311034', 0.0),
 ('NCBI:687071', 0.0),
 ('NCBI:1368960', 0.0),
 ('NCBI:581749', 0.0),
 ('NCBI:2340107', 0.0),
 ('NCBI:1368878', 0.0),
 ('NCBI:1765127', 0.0),
 ('NCBI:1368879', 0.

In [19]:
#visualisasi data
# mengecek BFS Degree tertinggi
for to_search  in contains_string_entire_column(df_node,'67753_Crinivirus').taxon_id.to_list():#[urutan[0][0]]:#["NCBI:7038"]:#"NCBI:33377","NCBI:7036",
    for edge in nx.bfs_edges(gnx.to_undirected(), source=to_search, depth_limit=1):
            s_id, o_id = edge

            s_label = gnx.nodes[s_id]['label'] +' '+s_id
            o_label = gnx.nodes[o_id]['label'] +' '+o_id
            o_grup = gnx.nodes[o_id]['group']

            # skip subjek to_search
            if (o_grup == 'serangga'):
                print(s_label,'-->', o_label)
    print('===============')

# Analisis Taksonomi

In [20]:
#5
# Ambil data NCBI
# data acuan
data_acuan=get_taxon_vector(acuan_,ncbi_ontology_url,False)
print(data_acuan)
data_ujian=get_taxon_vector(ujian_,ncbi_ontology_url)
print(data_ujian)

[('family', 'NCBI:30102_Cicadellidae'), ('order', 'NCBI:7524_Hemiptera'), ('class', 'NCBI:50557_Insecta'), ('phylum', 'NCBI:6656_Arthropoda'), ('kingdom', 'NCBI:33208_Metazoa'), ('superkingdom', 'NCBI:2759_Eukaryota')]
[('spesies', 'NCBI:30127_Graminella nigrifrons'), ('genus', 'NCBI:30125_Graminella <bugs>'), ('famili', 'NCBI:30102_Cicadellidae'), ('ordo', 'NCBI:7524_Hemiptera'), ('kelas', 'NCBI:50557_Insecta'), ('filum', 'NCBI:6656_Arthropoda'), ('kingdom', 'NCBI:33208_Metazoa'), ('superkingdom', 'NCBI:2759_Eukaryota')]


In [21]:
import importlib, sys
importlib.reload(sys.modules['modul.visualisasiHelper'])
from modul.visualisasiHelper import embeddingPlot,plotly_graph

#6 #konversi node networkx ke RDF
# input : df_node, URL, data_acuan
URL = "http://pyRDF2Vec"
df_serangga = df_node[df_node['group']=="serangga"]
CUSTOM_KG = df_serangga_to_rdf(df_serangga, URL, data_acuan)

#7 #embedding
list_serangga=df_node[df_node['group']=="serangga"].taxon_id.to_list()
# list entity yang akan diembedd. serangga acuan urutan terakhir
list_of_entities = [ URL+"#"+taxon_id for taxon_id in list_serangga ]
list_of_entities.append(f"{URL}#SERANGGA_ACUAN")
transformer, embeddings, _ = rdf_KG_to_embeddings(CUSTOM_KG, list_of_entities)
# list_of_entities == transformer._entities # True # artinya sama dua2nya

# dictionary serangga
dictionary_serangga = df_to_dictionary_taxon(df_serangga)
# output
# embeddings, list_of_entities, dictionary_serangga



# visualisasi
# input
# embeddings, list_of_entities, dictionary_serangga
embeddingPlot(embeddings, list_of_entities, dictionary_serangga, 650, 650)

In [22]:
#8
#euclidean distance

# buat dataframe
data_to_count=pd.DataFrame(embeddings, columns=list(range(0,100)))

# buat kolom label
ent=[data['label'] for index,data in gnx.nodes(data=True) if(data['group']=='serangga')] #jika serangga
ent.append("#SERANGGA_ACUAN")
data_to_count['label']=ent

#buat kolom entity
data_to_count['entity']=[i.replace("http://pyRDF2Vec#","") for i in transformer._entities]

# buat kolom hasil dc
for idx,row in data_to_count.iterrows(): #jika serangga acuan maka DC di isi nilai 1
    data_to_count.loc[idx,['dc_result']] = results_dc[row['entity']] if(row['entity']!="SERANGGA_ACUAN") else 1

#ambil koordinat acuan
acuan=next(data_to_count[data_to_count['label']=='#SERANGGA_ACUAN'].iterrows())[1]
acuan=np.array(tuple(acuan[i] for i in range(0,100)))
acuan

#hitung ED
for idx, row in data_to_count.iterrows():
    temp = np.array(tuple(row[i] for i in range(0,100)))
    data_to_count.loc[idx,['ed_result']] = np.linalg.norm(temp - acuan)

#drop data acuan
data_to_count.drop(data_to_count[data_to_count.label=="#SERANGGA_ACUAN"].index,inplace=True)

In [23]:
#drop kolom embedding
data_to_count.drop(columns=list(range(0,100)), inplace=True)

In [83]:
# simple scaling ed_result
# data_to_count["ed_result_scaled"] = data_to_count["ed_result"] / data_to_count["ed_result"].max()
data_to_count['ed_result_scaled'] = minmax(data_to_count['ed_result'])


# scaling dc 
# # dengan std
# data_to_count['dc_result_scaled'] = data_to_count['dc_result'] - data_to_count['dc_result'].mode()[0]
# # data_to_count['dc_result_scaled'] = data_to_count['dc_result']
# # lagi dengan minmax
# data_to_count["dc_result_scaled"] = data_to_count["dc_result_scaled"] / data_to_count["dc_result_scaled"].max()

# Perform scaling using standard deviation
# data_to_count['dc_result_scaled'] = std_scale(data_to_count['dc_result'])
# data_to_count['dc_result_scaled'] = minmax(data_to_count['dc_result_scaled'])
if data_to_count['dc_result'].max() == data_to_count['dc_result'].min():
    data_to_count['dc_result_scaled'] = data_to_count['dc_result']
else:
    data_to_count['dc_result_scaled'] = minmax(data_to_count['dc_result'])

In [84]:
data_to_count

,label,entity,dc_result,ed_result,ed_result_scaled,dc_result_scaled,result
0,Lygus lineolaris,NCBI:50650,0.0,1.200954,0.000000,0.0,NaN
1,Laodelphax striatella,NCBI:195883,0.0,1.238039,0.011537,0.0,NaN
2,Sitobion akebiae,NCBI:44662,0.0,1.400975,0.062224,0.0,NaN
3,Myzus persicae,NCBI:13164,0.0,1.413203,0.066028,0.0,NaN
4,Rhopalosiphum maidis,NCBI:43146,0.0,1.437696,0.073648,0.0,NaN
...,...,...,...,...,...,...,...
489,Liriomyza brassicae,NCBI:341052,0.0,4.095159,0.900349,0.0,NaN
490,Liriomyza huidobrensis,NCBI:127405,0.0,4.112128,0.905627,0.0,NaN
491,Liriomyza sativae,NCBI:127406,0.0,4.123224,0.909079,0.0,NaN
492,Campion,NCBI:1548265,0.0,4.244061,0.946670,0.0,NaN


In [85]:
#9
#hitung kombinasi
# for idx, row in data_to_count.iterrows():
#     _dc = row['dc_result_scaled']
#     _ed=( (row['ed_result_scaled']) if row['ed_result_scaled']!=0 else 1)
#     data_to_count.loc[idx,['result']] = _dc/_ed

data_to_count['result'] = (1+data_to_count['dc_result_scaled']) / (1+data_to_count['ed_result_scaled'])

# simple scaling result (final/kombinasi)
data_to_count['result'] = data_to_count['result'] / data_to_count['result'].max()

# Pengujian

In [86]:
# DC
data_to_count=data_to_count.sort_values('dc_result',ascending=False).reset_index(drop=True)
data_to_count[['label','entity','dc_result']]

,label,entity,dc_result
0,Lygus lineolaris,NCBI:50650,0.0
1,Corcyra cephalonica,NCBI:139035,0.0
2,Hypsipyla,NCBI:181329,0.0
3,Andrena pilipes,NCBI:1126407,0.0
4,Pelopidas agna,NCBI:218748,0.0
...,...,...,...
489,Sesamia,NCBI:134396,0.0
490,Pyrausta,NCBI:582143,0.0
491,Spodoptera dolichos,NCBI:134404,0.0
492,Scirpophaga,NCBI:72365,0.0


In [87]:

# Pengujian dc
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:50650_Lygus lineolaris', 'genus': 'NCBI:30084_Lygus', 'famili': 'NCBI:30083_Miridae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25
hasil  {'spesies': 'NCBI:139035_Corcyra^species', 'genus': 'NCBI:139035_Corcyra', 'famili': 'NCBI:7135_Pyralidae', 'ordo': 'NCBI:7088_Lepidoptera'}
0.0
hasil  {'spesies': 'NCBI:181329_Hypsipyla^species', 'genus': 'NCBI:181329_Hypsipyla', 'famili': 'NCBI:7135_Pyralidae', 'ordo': 'NCBI:7088_Lepidoptera'}
0.0


In [88]:
# ED
data_to_count=data_to_count.sort_values('ed_result',ascending=True).reset_index(drop=True)
data_to_count[['label','entity','ed_result']]

,label,entity,ed_result
0,Lygus lineolaris,NCBI:50650,1.200954
1,Laodelphax striatella,NCBI:195883,1.238039
2,Sitobion akebiae,NCBI:44662,1.400975
3,Myzus persicae,NCBI:13164,1.413203
4,Rhopalosiphum maidis,NCBI:43146,1.437696
...,...,...,...
489,Liriomyza brassicae,NCBI:341052,4.095159
490,Liriomyza huidobrensis,NCBI:127405,4.112128
491,Liriomyza sativae,NCBI:127406,4.123224
492,Campion,NCBI:1548265,4.244061


In [89]:
# Pengujian ed
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:50650_Lygus lineolaris', 'genus': 'NCBI:30084_Lygus', 'famili': 'NCBI:30083_Miridae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25
hasil  {'spesies': 'NCBI:195883_Laodelphax striatellus', 'genus': 'NCBI:195882_Laodelphax', 'famili': 'NCBI:33362_Delphacidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25
hasil  {'spesies': 'NCBI:44662_Sitobion^species', 'genus': 'NCBI:44662_Sitobion', 'famili': 'NCBI:27482_Aphididae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25


In [90]:
# final score kombinasi
data_to_count=data_to_count.sort_values('result',ascending=False).reset_index(drop=True)
data_to_count#[['label','dc_result','ed_result','result']]

,label,entity,dc_result,ed_result,ed_result_scaled,dc_result_scaled,result
0,Lygus lineolaris,NCBI:50650,0.0,1.200954,0.000000,0.0,1.000000
1,Laodelphax striatella,NCBI:195883,0.0,1.238039,0.011537,0.0,0.988595
2,Sitobion akebiae,NCBI:44662,0.0,1.400975,0.062224,0.0,0.941421
3,Myzus persicae,NCBI:13164,0.0,1.413203,0.066028,0.0,0.938062
4,Rhopalosiphum maidis,NCBI:43146,0.0,1.437696,0.073648,0.0,0.931404
...,...,...,...,...,...,...,...
489,Liriomyza brassicae,NCBI:341052,0.0,4.095159,0.900349,0.0,0.526219
490,Liriomyza huidobrensis,NCBI:127405,0.0,4.112128,0.905627,0.0,0.524762
491,Liriomyza sativae,NCBI:127406,0.0,4.123224,0.909079,0.0,0.523813
492,Campion,NCBI:1548265,0.0,4.244061,0.946670,0.0,0.513698


In [91]:
# Pengujian kombinasi
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:50650_Lygus lineolaris', 'genus': 'NCBI:30084_Lygus', 'famili': 'NCBI:30083_Miridae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25
hasil  {'spesies': 'NCBI:195883_Laodelphax striatellus', 'genus': 'NCBI:195882_Laodelphax', 'famili': 'NCBI:33362_Delphacidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25
hasil  {'spesies': 'NCBI:44662_Sitobion^species', 'genus': 'NCBI:44662_Sitobion', 'famili': 'NCBI:27482_Aphididae', 'ordo': 'NCBI:7524_Hemiptera'}
0.25


In [92]:
# Pengujian kombinasi
# cek=1
# takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
# urutan=0
# while cek/len(takson)>0:
#     id_hasil=data_to_count.iloc[urutan].entity
#     cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
#     cek_ujian= { k:v for k,v in data_ujian if k in takson }
#     # print(acuan_,'->', data_)
#     # print('ujian ',cek_ujian)
#     print('hasil ', urutan ,cek_hasil)
#     cek=0
#     for i in reversed(takson):
#         cekk=cek_hasil[i]==cek_ujian[i]
#         cek+=cekk
#         # print(i, cekk)
#     print(cek/len(takson))
#     urutan+=1

# Visualisasi Hasil Analisis

In [93]:
import plotly.graph_objects as go

to_itter=data_to_count.sort_values('dc_result',ascending=False).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('dc_result',ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.dc_result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='Degree centrality',
    xaxis_title='Degree',
    yaxis_title='Insect'
)

fig.show()

In [94]:
to_itter=data_to_count.sort_values('ed_result',ascending=True).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('ed_result',ascending=False).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.ed_result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='Euclidean distance',
    xaxis_title='Distance',
    yaxis_title='Insect'
)

fig.show()

In [95]:
import plotly.graph_objects as go

to_itter=data_to_count.sort_values('result',ascending=False).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('result',ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='final score',
    xaxis_title='Score',
    yaxis_title='Insect'
)

fig.show()

In [96]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the color for all bars
bar_color = '#636EFA'

# First plot
to_itter = data_to_count.sort_values('dc_result', ascending=False).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('dc_result', ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.dc_result)

fig = make_subplots(rows=2, cols=2, subplot_titles=('(a) Degree centrality', '(b) Euclidean distance', '(c) Final score', ''), vertical_spacing=0.1, horizontal_spacing=0.15)
fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=1, col=1)

# Second plot
to_itter = data_to_count.sort_values('ed_result', ascending=True).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('ed_result', ascending=False).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.ed_result)

fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=1, col=2)

# Third plot
to_itter = data_to_count.sort_values('result', ascending=False).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('result', ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.result)

fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=2, col=1)

# Hide the empty subplot
fig.update_layout(showlegend=False)
fig.update_annotations(visible=False, selector="title.text")

# Update layout for subplot titles
fig.update_layout(
    # title_text="Combined Plots",
    title_font={'size':24},
    title_x=0.5,
    title_y=0.95,
    title_xanchor='center',
    title_yanchor='top'
)

# Rotate y-axis labels
fig.update_yaxes(tickangle=45)

fig.update_layout(height=800, width=1200)
fig.show()


# Dibawah ini tidak termasuk

In [97]:
# #visualisasi data
# # mengecek BFS Degree tertinggi
# for edge in nx.bfs_edges(gnx.to_undirected(), source="NCBI:7038", depth_limit=1):
#         s_id, o_id = edge

#         s_label = gnx.nodes[s_id]['label'] +' '+s_id
#         o_label = gnx.nodes[o_id]['label'] +' '+o_id
#         o_grup = gnx.nodes[o_id]['group']

#         # skip subjek to_search

#         print(s_label,'-->', o_label)
# print('===============')

In [98]:
df_node[df_node['taxon_name'].str.contains('Thrips')]

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_rank,group,color,virus_utama
321,NCBI:1350418,Thrips parvispinus,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:1350418_Thrips parvispinus,NaN,serangga,#b22222,NaN
322,NCBI:407017,Thrips nigropilosus,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:407017_Thrips nigropilosus,NaN,serangga,#b22222,NaN
323,NCBI:163894,Thrips hawaiiensis,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:163894_Thrips hawaiiensis,NaN,serangga,#b22222,NaN


In [99]:
df_edge.interaction_type.unique()

array(['pathogenOf', 'hasHost', 'visitFlowersOf', 'pollinates', 'visit'],
      dtype=object)

In [100]:
ini=df_edge[df_edge['interaction_type'].isin(['visitFlowersOf'])]
ini

,source_taxon_id,target_taxon_id,interaction_type
172,NCBI:178013,NCBI:4530,visitFlowersOf
173,NCBI:60900,NCBI:4530,visitFlowersOf
174,NCBI:2008740,NCBI:4530,visitFlowersOf
204,NCBI:2939337,NCBI:3708,visitFlowersOf
205,NCBI:1126407,NCBI:3708,visitFlowersOf
...,...,...,...
671,NCBI:156310,NCBI:3711,visitFlowersOf
672,NCBI:1955922,NCBI:3711,visitFlowersOf
673,NCBI:598969,NCBI:3711,visitFlowersOf
674,NCBI:930006,NCBI:3711,visitFlowersOf


In [101]:
df_node[df_node['taxon_id'].isin(
    ini.source_taxon_id.to_list()+
    ini.target_taxon_id.to_list()
)]

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_rank,group,color,virus_utama
0,NCBI:4530,Oryza sativa,Plantae | Tracheophyta | Liliopsida | Poales |...,GBIF:6 | NCBI:58023 | NCBI:4447 | NCBI:38820 |...,kingdom | phylum | class | order | family | ge...,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:38820_Poales,NCBI:4479_Poaceae,NCBI:4527_Oryza,NCBI:4530_Oryza sativa,NaN,tanaman,#1f922b,NaN
1,NCBI:4577,Zea mays,Plantae | Tracheophyta | Liliopsida | Poales |...,GBIF:6 | NCBI:58023 | NCBI:4447 | NCBI:38820 |...,kingdom | phylum | class | order | family | ge...,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:38820_Poales,NCBI:4479_Poaceae,NCBI:4575_Zea,NCBI:4577_Zea mays,NaN,tanaman,#1f922b,NaN
164,NCBI:60900,Dactylurina staudingeri,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7458_Apidae,NCBI:60899_Dactylurina,NCBI:60900_Dactylurina staudingeri,NaN,serangga,#b22222,NaN
165,NCBI:2008740,Sceliphron madraspatanum,root | cellular organisms | Eukaryota | Opisth...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33154 ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7485_Sphecidae,NCBI:161220_Sceliphron,NCBI:2008740_Sceliphron madraspatanum,NaN,serangga,#b22222,NaN
191,NCBI:30195,Bombus terrestris,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7458_Apidae,NCBI:28641_Bombus <genus>,NCBI:30195_Bombus terrestris,NaN,serangga,#b22222,NaN
192,NCBI:2939337,Bombyliidae sp.,root | cellular organisms | Eukaryota | Opisth...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33154 ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7147_Diptera,NCBI:50674_Bombyliidae,NCBI:50674_Bombyliidae^genus,NCBI:2939337_Bombyliidae sp.,NaN,serangga,#b22222,NaN
193,NCBI:1126407,Andrena pilipes,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:48719_Andrenidae,NCBI:48720_Andrena <genus>,NCBI:1126407_Andrena pilipes,NaN,serangga,#b22222,NaN
194,NCBI:395521,Bombus distinguendus,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7458_Apidae,NCBI:28641_Bombus <genus>,NCBI:395521_Bombus distinguendus,NaN,serangga,#b22222,NaN
195,NCBI:30193,Bombus lucorum,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7458_Apidae,NCBI:28641_Bombus <genus>,NCBI:30193_Bombus lucorum,NaN,serangga,#b22222,NaN
196,NCBI:601640,Nomada lathburiana,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7458_Apidae,NCBI:95295_Nomada,NCBI:601640_Nomada lathburiana,NaN,serangga,#b22222,NaN


In [102]:
contains_string_entire_column(df_edge,'NCBI:1341303')

,source_taxon_id,target_taxon_id,interaction_type


In [103]:
contains_string_entire_column(df_node,'NCBI:12315')

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_rank,group,color,virus_utama
